In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [2]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        #List out our bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal.
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits)) #Returns a random state for each episode.
        return self.state
        
    def pullArm(self,action):
        #Get a random number.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            #return a positive reward.
            return 1
        else:
            #return a negative reward.
            return -1

In [3]:
class agent():
    # defined for one bandit, not 3
    def __init__(self, lr, s_size,a_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[1],dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
        output = slim.fully_connected(state_in_OH,a_size,\
            biases_initializer=None,activation_fn=tf.nn.sigmoid,weights_initializer=tf.ones_initializer())
        self.output = tf.reshape(output,[-1])
        self.chosen_action = tf.argmax(self.output,0)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [11]:
tf.reset_default_graph() #Clear the Tensorflow graph.

cBandit = contextual_bandit() #Load the bandits.
myAgent = agent(lr=0.001,s_size=cBandit.num_bandits,a_size=cBandit.num_actions) #Load the agent.
weights = tf.trainable_variables()[0] #The weights we will evaluate to look into the network.

total_episodes = 10000 #Set total number of episodes to train agent on.
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions]) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit() #Get a state from the environment.
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            output, action = sess.run([myAgent.output, myAgent.chosen_action], feed_dict={myAgent.state_in:[s]})
            print(output, action)
        
        reward = cBandit.pullArm(action) #Get our reward for taking an action given a bandit.
        
        #Update the network.
        feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update,weights], feed_dict=feed_dict)
        
        #Update our running tally of scores.
        total_reward[s,action] += reward
        if i % 500 == 0:
            print ("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))        
        i+=1
        # print(ww)
for a in range(cBandit.num_bandits):
    print ("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print ("...and it was right!")
    else:
        print ("...and it was wrong!")

[0.7310586 0.7310586 0.7310586 0.7310586] 0
Mean reward for each of the 3 bandits: [-0.25  0.    0.  ]
[0.7310586 0.7310586 0.7310586 0.7310586] 0
[0.7311114 0.7310586 0.7310586 0.7310586] 0
[0.7310586 0.7310586 0.7310586 0.7310586] 0
[0.7310057 0.7310586 0.7310586 0.7310586] 1
[0.7310057 0.7310586 0.7310586 0.7310586] 1
[0.73116434 0.7310586  0.7310586  0.7310586 ] 0
[0.7309528 0.7310057 0.7310586 0.7310586] 2
[0.7309528 0.7310057 0.7311114 0.7310586] 2
[0.73121715 0.7310586  0.7310586  0.7310586 ] 0
[0.7310057 0.7311114 0.7310586 0.7310586] 1
[0.7310057  0.73116434 0.7310586  0.7310586 ] 1
[0.73132277 0.7310586  0.7310586  0.7310586 ] 0
[0.7310057  0.73121715 0.7310586  0.7310586 ] 1
[0.7310057 0.73127   0.7310586 0.7310586] 1
[0.7313756 0.7310586 0.7310586 0.7310057] 0
[0.7310057  0.73132277 0.7310586  0.7310586 ] 1
[0.7309528  0.7310057  0.73116434 0.7310586 ] 2
[0.7309528  0.7310057  0.73121715 0.7310586 ] 2
[0.7309528 0.7310057 0.73127   0.7310586] 2
[0.7309528  0.7310057  0.7313

[0.7342531 0.7310586 0.7308999 0.7309528] 0
[0.7308999  0.7310057  0.7314813  0.73116434] 2
[0.7343049 0.7310586 0.7308999 0.7309528] 0
[0.73435676 0.7310586  0.7308999  0.7309528 ] 0
[0.7310586  0.73477083 0.7310057  0.7310586 ] 1
[0.73440856 0.7310586  0.7308999  0.7309528 ] 0
[0.7310586 0.7348225 0.7310057 0.7310586] 1
[0.7310586 0.7348742 0.7310057 0.7310586] 1
[0.73084694 0.7310057  0.73142856 0.73116434] 2
[0.7310586 0.7349258 0.7310057 0.7310586] 1
[0.73084694 0.7310057  0.7313758  0.73116434] 2
[0.73446035 0.7310586  0.7308999  0.7309528 ] 0
[0.7310586 0.7349775 0.7310057 0.7310586] 1
[0.7345122 0.7310586 0.7308999 0.7309528] 0
[0.73084694 0.7309528  0.73142856 0.73116434] 2
[0.7310586 0.7350291 0.7310057 0.7310586] 1
[0.73084694 0.7309528  0.7314814  0.73116434] 2
[0.7310586 0.7350807 0.7310057 0.7310586] 1
[0.7345639 0.7310586 0.7308999 0.7309528] 0
[0.7310586 0.7351322 0.7310057 0.7310586] 1
[0.7346157 0.7310586 0.7308999 0.7309528] 0
[0.7346674 0.7310586 0.7308999 0.7309528

[0.73111147 0.7374876  0.73079395 0.7311114 ] 1
[0.73084694 0.73084694 0.7312704  0.7334212 ] 3
[0.73111147 0.7375384  0.73079395 0.7311114 ] 1
[0.73111147 0.73758924 0.73079395 0.7311114 ] 1
[0.73111147 0.7376908  0.73079395 0.7311114 ] 1
[0.73084694 0.73084694 0.7312704  0.7334733 ] 3
[0.73111147 0.73774153 0.73079395 0.7311114 ] 1
[0.73084694 0.73084694 0.7312704  0.7335254 ] 3
[0.7376908 0.7310057 0.730741  0.7308999] 0
[0.73111147 0.73779225 0.73079395 0.7311114 ] 1
[0.73774153 0.7310057  0.730741   0.7308999 ] 0
[0.73084694 0.73084694 0.7312704  0.7335775 ] 3
[0.73111147 0.73784304 0.73079395 0.7311114 ] 1
[0.73779225 0.7310057  0.730741   0.7308999 ] 0
[0.73111147 0.7378937  0.73079395 0.7311114 ] 1
[0.73084694 0.73084694 0.7312704  0.7336296 ] 3
[0.73784304 0.7310057  0.730741   0.7308999 ] 0
[0.7378937 0.7310057 0.730741  0.7308999] 0
[0.73084694 0.73084694 0.7312704  0.7336816 ] 3
[0.73084694 0.73084694 0.7312704  0.73373365] 3
[0.7379444 0.7310057 0.730741  0.7308999] 0
[0.7

[0.73084694 0.7308999  0.73121756 0.7367744 ] 3
[0.7311644 0.7404585 0.730741  0.7310586] 1
[0.7410062  0.7309528  0.73068804 0.7308999 ] 0
[0.7311644 0.7405083 0.730741  0.7310586] 1
[0.7311644  0.74055815 0.730741   0.7310586 ] 1
[0.73084694 0.7308999  0.73121756 0.7368254 ] 3
[0.73084694 0.7308999  0.73121756 0.7368765 ] 3
[0.7311644  0.74060804 0.730741   0.7310586 ] 1
[0.74110556 0.7309528  0.73068804 0.7308999 ] 0
[0.74115527 0.7309528  0.73068804 0.7308999 ] 0
[0.7311644  0.74065787 0.730741   0.7310586 ] 1
[0.73084694 0.7308999  0.73121756 0.7369275 ] 3
[0.73084694 0.7308999  0.73121756 0.73697853] 3
[0.73084694 0.7308999  0.73121756 0.73702943] 3
[0.7412049  0.7309528  0.73068804 0.7308999 ] 0
[0.7311644  0.74070764 0.730741   0.7310586 ] 1
[0.73084694 0.7308999  0.73121756 0.73708045] 3
[0.73084694 0.7308999  0.73121756 0.73713136] 3
[0.7311644  0.74075747 0.730741   0.7310586 ] 1
[0.73084694 0.7308999  0.73121756 0.7371823 ] 3
[0.74125457 0.7309528  0.73068804 0.7308999 ] 0


[0.73084694 0.7311644  0.7311647  0.7398587 ] 3
[0.73084694 0.7311644  0.7311647  0.73990875] 3
[0.74444765 0.7309528  0.73058206 0.7308999 ] 0
[0.73084694 0.7311644  0.7311647  0.7399588 ] 3
[0.7444963  0.7308999  0.73058206 0.7308999 ] 0
[0.73111147 0.7437655  0.73068804 0.7308999 ] 1
[0.73084694 0.7311644  0.7311647  0.7400588 ] 3
[0.73084694 0.7311644  0.7311647  0.7401089 ] 3
[0.73111147 0.7438143  0.73068804 0.7308999 ] 1
[0.74454486 0.7308999  0.73058206 0.73084694] 0
[0.73084694 0.7311644  0.7311647  0.74015886] 3
[0.73084694 0.7311644  0.7311647  0.7402088 ] 3
[0.7445935  0.7308999  0.73058206 0.73084694] 0
[0.73084694 0.7311644  0.7311647  0.74025875] 3
[0.73111147 0.7438631  0.73068804 0.7308999 ] 1
[0.73084694 0.7311644  0.7311647  0.7403087 ] 3
[0.7446421  0.7308999  0.73058206 0.73084694] 0
[0.73111147 0.7439119  0.730741   0.7308999 ] 1
[0.7446906  0.7308999  0.73058206 0.73084694] 0
[0.73111147 0.7439607  0.730741   0.7308999 ] 1
[0.73111147 0.74400944 0.730741   0.7308

...

[0.7304764  0.82844067 0.72828627 0.73005134] 1
[0.82972443 0.72655684 0.7270991  0.72623056] 0
[0.73042333 0.8284894  0.72828627 0.73005134] 1
[0.7306354  0.73095316 0.7310594  0.8283431 ] 3
[0.8297485  0.72655684 0.7270991  0.72623056] 0
[0.7306354  0.73095316 0.7310594  0.82836753] 3
[0.7306354  0.73095316 0.7310594  0.8283919 ] 3
[0.73042333 0.8285138  0.72828627 0.73005134] 1
[0.7306354  0.73095316 0.7310594  0.8284163 ] 3
[0.82977253 0.72655684 0.7270991  0.72623056] 0
[0.7306354  0.73095316 0.7310594  0.82844067] 3
[0.7306354  0.73095316 0.7310594  0.82846504] 3
[0.73042333 0.8285382  0.72828627 0.73005134] 1
[0.7306354  0.73095316 0.7310594  0.8284894 ] 3
[0.8297966  0.72655684 0.7270991  0.72623056] 0
[0.73042333 0.8285626  0.72828627 0.73005134] 1
[0.73042333 0.8285869  0.72828627 0.73005134] 1
[0.73042333 0.8286112  0.72828627 0.73005134] 1
[0.7306354  0.73095316 0.7310594  0.8285138 ] 3
[0.73042333 0.8286356  0.72828627 0.73005134] 1
[0.73042333 0.8286599  0.72828627 0.7300

[0.73047644 0.8301565  0.7281787  0.7299981 ] 1
[0.73063546 0.73084736 0.7310065  0.8298687 ] 3
[0.83125186 0.72639376 0.727045   0.72617614] 0
[0.73042333 0.8301804  0.7281787  0.7299981 ] 1
[0.73063546 0.73084736 0.7310594  0.82989264] 3
[0.73063546 0.73084736 0.7310594  0.8299167 ] 3
[0.8312755  0.72639376 0.727045   0.72617614] 0
[0.73063546 0.73084736 0.7310594  0.8299407 ] 3
[0.8312991  0.72639376 0.727045   0.72617614] 0
[0.7303703 0.8302044 0.7281787 0.7299981] 1
[0.8313228  0.72639376 0.727045   0.72617614] 0
[0.7303703 0.8302283 0.7281787 0.7299981] 1
[0.7303703  0.83025223 0.7281787  0.7299981 ] 1
[0.7303703  0.83027613 0.7281787  0.7299981 ] 1
[0.7303703 0.8303001 0.7281787 0.7299981] 1
[0.83134645 0.72639376 0.727045   0.72617614] 0
[0.7303703 0.830324  0.7281787 0.7299981] 1
[0.83137006 0.7263394  0.727045   0.72617614] 0
[0.7303703 0.8303479 0.7281787 0.7299981] 1
[0.73063546 0.73084736 0.7310065  0.82996476] 3
[0.8313937  0.7263394  0.727045   0.72617614] 0
[0.7303703 0

[0.73063546 0.7307414  0.7308478  0.8314174 ] 3
[0.73063546 0.7307414  0.7308478  0.83144104] 3
[0.83277804 0.72628504 0.727045   0.72606725] 0
[0.73063546 0.7307414  0.7308478  0.8314646 ] 3
[0.8328013  0.72628504 0.727045   0.72606725] 0
[0.7303171  0.8318418  0.72807103 0.7299448 ] 1
[0.73063546 0.7307414  0.7308478  0.83148825] 3
[0.8328246  0.72628504 0.727045   0.72606725] 0
[0.73063546 0.7307414  0.7308478  0.8315119 ] 3
[0.7303171  0.8318653  0.72807103 0.7299448 ] 1
[0.7303171  0.8318888  0.72807103 0.7299448 ] 1
[0.73063546 0.7307414  0.7308478  0.83153546] 3
[0.83284783 0.72628504 0.7269909  0.72606725] 0
[0.7303171  0.83191234 0.72807103 0.7299448 ] 1
[0.73063546 0.7307414  0.7308478  0.83155906] 3
[0.73063546 0.7307414  0.7308478  0.8315826 ] 3
[0.7303171  0.8319358  0.72807103 0.7299448 ] 1
[0.7306885 0.7307414 0.7308478 0.8316062] 3
[0.83287114 0.72628504 0.7269909  0.72606725] 0
[0.8328944  0.72628504 0.7269909  0.72606725] 0
[0.8329176  0.72628504 0.7269909  0.72606725

[0.8343024  0.72623056 0.72688246 0.7260127 ] 0
[0.8343253  0.72623056 0.72688246 0.7260127 ] 0
[0.7303171  0.83340466 0.7280172  0.7298384 ] 1
[0.7303171  0.83342785 0.7280172  0.7298384 ] 1
[0.7305825 0.7307414 0.7308478 0.8331499] 3
[0.7305825  0.7307414  0.7308478  0.83317304] 3
[0.8343482  0.72623056 0.72688246 0.7260127 ] 0
[0.7305825 0.7307414 0.7308478 0.8331962] 3
[0.83437115 0.72623056 0.72688246 0.7260127 ] 0
[0.83439404 0.72623056 0.72688246 0.7260127 ] 0
[0.7303171 0.833451  0.7280172 0.7298384] 1
[0.7303171  0.83347404 0.7280172  0.7298384 ] 1
[0.7303171  0.83349717 0.7280172  0.7298384 ] 1
[0.7305825 0.7307414 0.7308478 0.8332194] 3
[0.7303171 0.8335203 0.7280172 0.7298384] 1
[0.7303171  0.83354336 0.7280172  0.7298384 ] 1
[0.7305825 0.7307414 0.7308478 0.8332426] 3
[0.7303171 0.8335665 0.7280172 0.7298384] 1
[0.7303171 0.8335895 0.7280172 0.7298384] 1
[0.7305825 0.7307414 0.7308478 0.8332658] 3
[0.7305825 0.7307414 0.7308478 0.8332889] 3
[0.8344169  0.72623056 0.7268824

[0.7303172 0.8348734 0.7279633 0.7298384] 1
[0.7305295  0.73068845 0.7308478  0.8345769 ] 3
[0.8360945  0.72590375 0.72688246 0.7260127 ] 0
[0.83611697 0.72590375 0.72688246 0.7260127 ] 0
[0.8361395  0.72590375 0.72688246 0.7260127 ] 0
[0.7305295  0.73068845 0.7308478  0.83459973] 3
[0.7303172  0.83489615 0.7279633  0.7298384 ] 1
[0.7303172 0.8349189 0.7279633 0.7298384] 1
[0.8361619  0.72590375 0.72688246 0.7260127 ] 0
[0.7303172 0.8349417 0.7279633 0.7298384] 1
[0.7303172  0.83496433 0.7279633  0.7298384 ] 1
[0.7303172 0.8349871 0.7279633 0.7298384] 1
[0.8361843  0.72590375 0.72688246 0.7260127 ] 0
[0.83620673 0.72590375 0.72688246 0.7260127 ] 0
[0.7305295  0.7307414  0.7307948  0.83462256] 3
[0.7305295  0.7307414  0.7307948  0.83464545] 3
[0.7303172 0.8350099 0.7279633 0.7298384] 1
[0.8362292  0.72590375 0.72688246 0.7260127 ] 0
[0.8362517  0.72590375 0.72688246 0.7260127 ] 0
[0.83627415 0.72590375 0.72688246 0.7260127 ] 0
[0.7303172 0.8350326 0.7279633 0.7298384] 1
[0.7303172 0.835